# 9. 객체, 케이스 클래스, 트레이트

## 객체
**구문: 객체 정의하기**

object <식별자> [extends <식별자>] [{필드, 메소드, 클래스}]

In [1]:
object Hello { println ("in Hello"); def hi = "hi"}

defined object Hello

In [2]:
println(Hello.hi)

in Hello
hi


In [3]:
println(Hello.hi)

hi


In [4]:
object HtmlUtils{
    def removeMarkup(input: String) = {
        input
            .replaceAll("""</?\w[^>]*>""", """""")
            .replaceAll("<.*","")
    }
}

defined object HtmlUtils

In [5]:
val html = "<html><body><h1>Introduction</h1></body></html>"

html: String = "<html><body><h1>Introduction</h1></body></html>"

In [6]:
val text = HtmlUtils.removeMarkup(html)

text: String = "Introduction"

## Apply 메소드와 동반 객체

:paste


// Entering paste mode

In [9]:
class Multiplier(val x: Int) { def product(y: Int) = x * y}
object Multiplier { def apply(x: Int) = new Multiplier(x)}
//Exiting paste mode

defined class Multiplier
defined object Multiplier

In [10]:
val tripler = Multiplier(3)

tripler: Multiplier = ammonite.$sess.cmd8$Helper$Multiplier@3e4a8523

In [11]:
val result = tripler.product(13)

result: Int = 39

In [14]:
object DBConnection {
    private val db_url = "jdbc://localhost"
    private val db_user = "franken"
    private val db_pass = "berry"
    
def apply() = new DBConnection
}

class DBConnection {
    private val props = Map (
    "url" -> DBConnection.db_url,
    "user" -> DBConnection.db_user,
    "pass" -> DBConnection.db_pass
    )
    println(s"Created new connection for " + props("url"))
}

defined object DBConnection
defined class DBConnection

In [15]:
val conn = DBConnection()

Created new connection for jdbc://localhost


conn: DBConnection = ammonite.$sess.cmd13$Helper$DBConnection@1be7e704

## 케이스 클래스

** 구문: 케이스 클래스 정의하기 **

case class <식별자> ([var] <식별자>: <타입> [, ...])  
....................[extends <식별자>(<입력 매개변수>)]  
....................[{ 필드와 메소드}]  

In [16]:
case class Character(name: String, isThief: Boolean)

defined class Character

In [17]:
val h = Character("Hadrian", true)

h: Character = Character("Hadrian", true)

In [18]:
val r = h.copy(name = "Royce")

r: Character = Character("Royce", true)

In [19]:
h == r

res18: Boolean = false

In [20]:
h match {
    case Character(x, true) => s"$x is a thief"
    case Character(x, flase) => s"$x is not a thief"
}

res19: String = "Hadrian is a thief"

## 트레이트

** 구문: 트레이트 정의하기 **

trait <식별자> [extends <식별자>] [{필드, 메소드, 클래스}]

In [21]:
trait HtmlUtils{
    def removeMarkup(input: String) = {
        input
            .replaceAll("""</?\w[^>]*>""", """""")
            .replaceAll("<.*","")
    }
}

defined trait HtmlUtils

In [22]:
class Page(val s: String) extends HtmlUtils{
    def asPlainText = removeMarkup(s)
}

defined class Page

In [23]:
new Page("<html><body><h1>Introduction</h1></body></html>").asPlainText

res22: String = "Introduction"

In [24]:
trait SafeStringUtils {
    def trimToNone(s: String): Option[String] = {
        Option(s) map(_.trim) filterNot(_.isEmpty)
    }
}

defined trait SafeStringUtils

In [25]:
class Page(val s: String) extends SafeStringUtils with HtmlUtils{
    def asPlainText: String = {
        trimToNone(s) map removeMarkup getOrElse "n/a"
    }
}

defined class Page

In [26]:
new Page("<html><body><h1>Introduction</h1></body></html>").asPlainText

res25: String = "Introduction"

In [27]:
new Page(" ").asPlainText

res26: String = "n/a"

In [28]:
new Page(null).asPlainText

res27: String = "n/a"

In [29]:
trait Base { override def toString = "Base"}

defined trait Base

In [30]:
class A extends Base { override def toString = "A->" + super.toString}

defined class A

In [33]:
trait B extends Base { override def toString = "B->" + super.toString}

defined trait B

In [34]:
trait C extends Base { override def toString = "C->" + super.toString}

defined trait C

In [35]:
class D extends A with B with C { override def toString = "D->" + super.toString}

defined class D

In [36]:
new D()

res35: D = D->C->B->A->Base

In [37]:
class RGBColor(val color: Int) { def hex = f"$color%06X" }

defined class RGBColor

In [38]:
val green = new RGBColor(255 << 8).hex

green: String = "00FF00"

In [39]:
trait Opaque extends RGBColor {override def hex = s"${super.hex}FF"}

defined trait Opaque

In [40]:
trait Sheer extends RGBColor {override def hex = s"${super.hex}33" }

defined trait Sheer

In [41]:
class Paint(color: Int) extends RGBColor(color) with Opaque

defined class Paint

In [42]:
class Overlay(color: Int) extends RGBColor(color) with Sheer

defined class Overlay

In [43]:
val red = new Paint(128 << 16).hex

red: String = "800000FF"

In [44]:
val blue = new Overlay(192).hex

blue: String = "0000C033"

## 셀프 타입

**구문: 셀프 타입 정의하기**  
trait .... { <식별자>: <타입> => ... }


In [45]:
class A {def hi = "hi"}

defined class A

In [46]:
trait B { self: A =>
override def toString = "B: "+ hi}

defined trait B

In [46]:
class C extends B

cmd46.sc:1: illegal inheritance;
 self-type Helper.this.C does not conform to cmd46.this.cmd45.B's selftype cmd46.this.cmd45.B with ammonite.$sess.cmd45.wrapper.cmd44.A
class C extends B
                ^Compilation Failed

: 

In [47]:
class C extends A with B

defined class C

In [48]:
new C()

res47: C = B: hi

In [50]:
import scala.util

import scala.util

In [49]:
class TestSuite(suiteName: String) { def start() {}}

defined class TestSuite

In [51]:
trait RandomSeeded {self: TestSuite => 
    def randomStart() {
        util.Random.setSeed(System.currentTimeMillis)
        self.start()
    }
}

defined trait RandomSeeded

In [52]:
class IdSpec extends TestSuite("ID Tests") with RandomSeeded {
    def testId() {println(util.Random.nextInt != 1)}
    override def start() { testId()}
    println("Starting...")
    randomStart()
}

defined class IdSpec

## 트레이트를 이용하여 인스턴스화

In [53]:
class A

defined class A

In [54]:
trait B {self: A =>}

defined trait B

In [55]:
val a = new A with B

a: A with B = ammonite.$sess.cmd54$Helper$$anon$1@1cb0e966

In [56]:
class User(val name: String) {
    def suffix =""
    override def toString = s"$name$suffix"
}

defined class User

In [57]:
trait Attorney {self: User => override def suffix = ", esq."}

defined trait Attorney

In [58]:
trait Wizard {self: User => override def suffix = ", Wizard."}

defined trait Wizard

In [59]:
trait Reverser {override def toString = super.toString.reverse}

defined trait Reverser

In [60]:
val h = new User("Harry P") with Wizard

h: User with Wizard = Harry P, Wizard.

In [61]:
val g = new User("Ginny W") with Attorney

g: User with Attorney = Ginny W, esq.

In [62]:
val l = new User("Luna L") with Wizard with Reverser

l: User with Wizard with Reverser = .draziW ,L anuL

## 인스턴스 구성원 임포트하기

In [63]:
case class Receipt(id: Int, amount: Double, who: String, title: String)

defined class Receipt

In [64]:
{
    val latteReceipt = Receipt(123, 4.12, "fred", "medium Latte")
    import latteReceipt._
    println(s"Sold a $title for $amount to $who")
}

Sold a medium Latte for 4.12 to fred


latteReceipt: Receipt = Receipt(123, 4.12, "fred", "medium Latte")
import latteReceipt._
    

In [66]:
import util.Random._

import util.Random._

In [67]:
import scala.util.Random._

import scala.util.Random._

In [68]:
import java.util.Random._

import java.util.Random._

In [69]:
val letters = alphanumeric.take(20).toList.mkString

letters: String = "vByOI2PoggPlxtGrRX4W"

In [71]:
val numbers = shuffle(1 to 20)

numbers: collection.immutable.IndexedSeq[Int] = Vector(
  2,
  8,
  5,
  7,
  16,
  14,
  6,
  4,
  19,
  15,
  9,
  20,
  13,
  18,
  17,
  10,
  12,
  1,
  11,
  3
)